In [ ]:
import pandas as pd
import numpy as np
import NUTILS as nutils
from sklearn.metrics import mean_squared_error, make_scorer
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from catboost import Pool, CatBoostRegressor

In [ ]:
df_t = pd.read_csv('/home/agi/Desktop/NOMAD/ensembling/train_full.csv')
df_s = pd.read_csv('/home/agi/Desktop/NOMAD/ensembling/test_full.csv')

X_train = nutils.drop_features(df_t)
X_submit = nutils.drop_features_s(df_s)
y_form = df_t["formation_energy_ev_natom"]
y_band = df_t["bandgap_energy_ev"]

In [ ]:
params = {
    'logging_level': 'Silent'
}

model_form = CatBoostRegressor(**params)
model_band = CatBoostRegressor(**params)

In [ ]:
k_fold = KFold(n_splits=10, random_state=7)
objective  = make_scorer(nutils.rmsle, greater_is_better=False)

scores_form = cross_val_score(model_form, X_train, y_form, scoring=objective, cv=k_fold)
scores_band = cross_val_score(model_band, X_train, y_band, scoring=objective, cv=k_fold)

rmse_scores = -scores_form - scores_band
nutils.display_scores(rmse_scores / 2)

In [ ]:
model_form.fit(X_train, y_form, cat_features=[0])
model_band.fit(X_train, y_band, cat_features=[0])

submit_pred_form = model_form.predict(X_submit)
submit_pred_band = model_band.predict(X_submit)

model_form.save_model('/home/agi/Desktop/NOMAD/ensembling/cat_models/cat_form_model1')
model_band.save_model('/home/agi/Desktop/NOMAD/ensembling/cat_models/cat_band_model1')

In [ ]:
# Build submission .csv
submission = np.concatenate((submit_pred_form.reshape(600,1), submit_pred_band.reshape(600,1)), axis=1)
submit_df = pd.DataFrame(submission, columns=['formation_energy_ev_natom', "bandgap_energy_ev"])
submit_df[submit_df < 0] = 0
submit_df.insert(0, 'id', range(1, 601))

# Save to file
submit_df.to_csv("/home/agi/Desktop/NOMAD/ensembling/subs/cat_1.csv", index=False)